In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
import torch.nn.functional as F
import math

Case Data

In [ ]:
 pip install PYPOWER #installing PyPower

     |████████████████████████████████| 348kB 9.1MB/s 


In [ ]:
from pypower.loadcase import loadcase
from pypower.makeYbus import makeYbus
import pypower
 



Accessing the Pmin and Pmax from the case and getting the slack bus:

In [ ]:
def Initialize(ppc):
  Gen = ppc["gen"]
  Bus =[]
  PMax= []
  PMin=[]
  i= 0 #iterator over every generator to get the Bus, Pmax , Pmin of the generator

  while (i!= len(Gen)):
    Bus.append(ppc["gen"][i,0])
    PMax.append(ppc["gen"][i,8])
    PMin.append(ppc["gen"][i,9])
    i = i+1

  Power= pd.DataFrame(columns=['Bus','Pmin','Pmax'])
  Power['Bus']= Bus
  Power['Pmin'] = PMin
  Power['Pmax'] = PMax 
  Power.set_index('Bus')
  #print(Power)
  #finding the slack bus

  slack = 0
  j=0 #iterator over every bus row to check for the bus type
  while(slack == 0):
    if(ppc["bus"][j,1]== 3):
      slack = ppc["bus"][j,0]
    j +=1
  #print(slack)
  slack_adapted = int(slack) - 1  
  return Power,slack, slack_adapted

Getting the admittance matrix:

In [ ]:
from pypower.ext2int import ext2int
from pypower.makeBdc import makeBdc
from scipy.sparse import csr_matrix

def GetAdmittance(ppc):
  ppc = ext2int(ppc)
  BInputbaseMVA, BInputbus, BInputbranch = ppc['baseMVA'], ppc['bus'],  ppc['branch']

  Bbus, Bf, Pbusinj, Pfinj = makeBdc(BInputbaseMVA, BInputbus, BInputbranch)
  Bbus = csr_matrix(Bbus).toarray() #convert the csr_matrix into an array 
  #print(Bbus)
  return Bbus


Importing the Pd, Pg and Va csv files:

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
def InitializeDataframes(): 
  PDdf = pd.read_csv('Pd.csv')
  PGdf = pd.read_csv('Pg.csv')
  VAdf = pd.read_csv('Va.csv')
  #Successdf = pd.read_csv('Success.csv')
  #print(PGdf)
  #print(PDdf)
  unNormalizedPD = PDdf
  unNormalizedPG = PGdf
  return PDdf, PGdf, VAdf, unNormalizedPD, unNormalizedPG 

In [ ]:
def GetMaxandMins(PDdf,slack_adapted):
  #Getting PD_maxs and PD_mins for later use
  PD_maxs = []
  PD_mins = []
  it =0
  for column in PDdf.columns:
          if(it!= slack_adapted):
            PD_maxs.append(PDdf[column].max()) 
            PD_mins.append(PDdf[column].min())
          it = it +1
  #print(PD_maxs)
  return PD_maxs, PD_mins

To Per Unit Function

In [ ]:
def To_Per_Unit(P, ppc):
  P = P /  ppc['baseMVA']
  return P

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
def To_Per_Unit_Torch(P,ppc):
  N= np.zeros((1,1))
  N[0][0] =  ppc['baseMVA']
  N = torch.from_numpy(N)
  PerUnit = torch.divide(P,N)
  return PerUnit


Load sampling and pre-processing

In [ ]:
def normalize(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        if(df_norm[column].max()!=0):
          df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm


*Linear transformation and mapping dimension reduction*


In [ ]:
def getslackindex(PGdf,slack):
  PGdf2=PGdf.copy()
  titles=list(PGdf2.columns)
  strslack=str(int(slack))
  temp=0
  t=0
  for i in range(len(titles)):
    if(titles[i]==strslack):
      temp1=titles[i]
      t=i
  return t


In [ ]:

def GetAlphafromPG(PGdf,Power):
  Alpha = PGdf.copy()
  i = 0
  for column in PGdf: 
    Alpha[column] = Alpha[column] - Power.at[i, "Pmin"]
    Alpha[column] = Alpha[column]/ (Power.at[i, "Pmax"]- Power.at[i, "Pmin"])
    Pmax=Power.at[i,"Pmax"]  
    print(Pmax)
    i= i+1    
  return Alpha

def GetAlphafromPGws(PGdf,Power,slack):
  t=getslackindex(PGdf,slack)
  print(PGdf)
  Alpha = PGdf.copy()
  Alpha = pd.DataFrame(Alpha)
  slackstring = str(int(slack))
  print(Alpha)
  Alpha = Alpha.drop(labels= slackstring , axis=1,inplace=False)
  i = 0
  for column in PGdf: 
    if(i == t):
      i = i+1
    else:
      Alpha[column] = Alpha[column] - Power.at[i, "Pmin"]
      Alpha[column] = Alpha[column]/ (Power.at[i, "Pmax"]- Power.at[i, "Pmin"])
      Pmax=Power.at[i,"Pmax"] 
      i= i+1  
  return Alpha


def GetPGfromAlpha(Alpha,Power, PGdf):
  i = 0
  PG = Alpha.copy()
  for column in PG:
    PG[column] = PG[column]* (Power.at[i, "Pmax"]- Power.at[i, "Pmin"])
    PG[column] = PG[column] + Power.at[i, "Pmin"]
    i= i+1
  return PG

def GetPGfromAlphaws(Alpha,Power,slack, PGdf):
  t=getslackindex(PGdf,slack)
  PG  = Alpha.copy()
  Pdiff = []
  Pmin = []
  i = 0
  for column in PG:
    if (i==t):
      i = i+1
    else:
      PG[column] = PG[column] * (Power.at[i, "Pmax"]- Power.at[i, "Pmin"])
      PG[column] = PG[column] + Power.at[i, "Pmin"]
      Pmax=Power.at[i,"Pmax"] 
      i= i+1
  return PG


Removing the slack bus from  PD , PG:

In [ ]:
def RemovingSlackFromPDPG(PDdf,PGdf,slack, unNormalizedPD):
  slackstring = str(int(slack))
  PDdf_ws=PDdf.drop(labels= slackstring , axis=1,inplace=False)
  #print(PDdf_ws) #PD without slack
  PGdf_ws=PGdf.drop(labels= slackstring , axis=1,inplace=False)
  #print(PGdf_ws) #PG without slack
  # PG NOT ALPHAC HERE CARFUL
  unNormalizedPD_ws = unNormalizedPD.drop(labels= slackstring , axis=1,inplace=False)
  #unNormalizedPG_ws = unNormalizedPG.drop(labels= slackstring , axis=1,inplace=False)
  return PDdf_ws, PGdf_ws, unNormalizedPD_ws




Removing the slack bus from admittance matrix:

In [ ]:
def RemovingSlack(slack_adapted, Bbus):
  B_tilda = np.delete(Bbus, slack_adapted,0) 
  B_tilda = np.delete(B_tilda, slack_adapted,1)
  return B_tilda 

Getting the Phase Angle (without the slack bus), for uploaded values

In [ ]:
def Get_B_Matrix_Inverse(slack_adapted,ppc):
  Bbus = GetAdmittance(ppc)
  B_tilda = RemovingSlack(slack_adapted, Bbus)
  #First need to invert the admittance matrix found previously
  B_inverse = np.linalg.inv(B_tilda) 
  return B_inverse 


In [ ]:
def Get_Phase_Angles(PG, unNormalizedPD_ws, slack_adapted,ppc):
  
  B_inverse= Get_B_Matrix_Inverse(slack_adapted,ppc)
  PDtemp = unNormalizedPD_ws.copy() # for this formula we need the unnormalized PD
  #preparing the PG matrix
  PG_full = pd.DataFrame(np.zeros(PDtemp.shape), columns = PDtemp.columns.tolist()) 
  PG_gen_list = PG.columns.tolist()
  for col in PG_full.columns:
    if col in PG_gen_list:
      PG_full[col].values[:] = PG[col].values[:] 

  #Preparing PD and PG
  PDtemp= To_Per_Unit(PDtemp,ppc) #turning PDs into per unit value
  PG_full= To_Per_Unit(PG_full,ppc) #turning the P generated values
  PDtemp = np.transpose(pd.DataFrame.to_numpy(PDtemp)) #transposing to have the buses as the rows and the cases as the columns
  PG_full= np.transpose(pd.DataFrame.to_numpy(PG_full)) #transposing to have the buses as the rows and the cases as the columns
  #getting the difference between PG and PD
  Difference = PG_full - PDtemp
 
  #getting the angles in degrees
  Angles =(180/math.pi)*np.matmul(B_inverse,Difference)
  
  return Angles
#Angles0 = Get_Phase_Angles(PGdf_ws, unNormalizedPD_ws)
#print(Angles0)

Getting the Phase Angle (with Slack)

In [ ]:
def getAnglesWithSlack(Angles):
  #We have to add 0s for the slack's phase angle 
  numberofrowsA , numberofcolumnsA= Angles.shape 
  numberofcolumnsA = numberofcolumnsA
  slackangles = [[0]*numberofcolumnsA]
  #print(slackangles)
  #np.append(Angles,slackangles,0)
  Angles = np.insert(Angles, int(slack_adapted) , slackangles , 0)
  return Angles
#Angles0 = getAnglesWithSlack(Angles0)
#print(Angles0)
#print(getAnglesWithSlack(Angles))


Loss Functions:
Getting LPG:


In [ ]:
#Penalty Function
def penalty(x):
  p = np.square(x) - 1
  return p

#A Matrix Function
def createAmatrix(ppc):
  n1, m1 = ppc['bus'].shape
  n2, m2 = ppc['branch'].shape
  A = np.empty((n2,n1))
  A[:] = 0
  Adf = pd.DataFrame(A)
  for row in range(len(Adf)):
    #print (ppc['branch'][row][0],row)
    #print (ppc['branch'][row][1],row)
    Adf[ppc['branch'][row][0]-1][row]= 1/ ((ppc['branch'][row][3]) * (To_Per_Unit(ppc['branch'][row][5],ppc))) #ai = 1/ xij Pijmax and ask prof jabr if the power in pu or angles in degrees or radiant 
    Adf[ppc['branch'][row][1]-1][row]= -1 / ((ppc['branch'][row][3]) * (To_Per_Unit(ppc['branch'][row][5],ppc))) #aj = - 1/ xij Pijmax 
    #print(Adf[ppc['branch'][row][0]-1][row])
    #print(Adf[ppc['branch'][row][1]-1][row])
  A = pd.DataFrame(Adf).to_numpy(dtype= 'float')
  return A


Preparing the data:

In [ ]:
def PreparingDnnData(PDdf_ws,PGdf,Power,slack):
  #drop the  0 values and you are left with 20 colums --> input to 1st layer like in the paper 
  import numpy as np
  import pandas as pd

  PDdf_withoutzeros= PDdf_ws.loc[:, (PDdf_ws != 0).any(axis=0)]
  #PDdf_withoutzeros

  Alpha = GetAlphafromPGws(PGdf, Power,slack)
  #Alpha

  inputsize = len(PDdf_withoutzeros.columns)
  outputsize = len(Alpha.columns)
  return PDdf_withoutzeros, Alpha, inputsize , outputsize 


In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as functional
def InitializeTrainTestData(test_size, PDdf_withoutzeros, Alpha):
  PD_train, PD_test, Alpha_train, Alpha_test = train_test_split(PDdf_withoutzeros, Alpha, test_size= test_size, random_state=42)
  return PD_train, PD_test, Alpha_train, Alpha_test

Initialize the network

Training data to tensors:

In [ ]:
def TrainingData(PD_train, Alpha_train):
  #INPUTS 
  #Convert to Array 
  inputs = PD_train.values
  inputs = inputs.astype ('float32') #emphasizing on the type

  #Convert into tensors
  inputs = torch.from_numpy(inputs)
  #print(inputs)

  #TARGETS
  #Convert to Array 
  Alpha_target = Alpha_train.values
  Alpha_target = Alpha_target.astype ('float32') #emphasizing on the type

  #Convert into tensors
  Alpha_target = torch.from_numpy(Alpha_target)
  #print(Alpha_target)
  return inputs, Alpha_target


Testing data to tensors:

In [ ]:
def TestingData(PD_test, Alpha_test):
  #INPUTS 
  #Convert to Array 
  inputs_test = PD_test.values
  inputs_test = inputs_test.astype ('float32') #emphasizing on the type

  #Convert into tensors
  inputs_test = torch.from_numpy(inputs_test)
  #print(inputs_test)

  #TARGETS
  #Convert to Array 
  Alpha_target_test = Alpha_test.values
  Alpha_target_test = Alpha_target_test.astype ('float32') #emphasizing on the type

  #Convert into tensors
  Alpha_target_test = torch.from_numpy(Alpha_target_test)
  #print(Alpha_target_test)
  return inputs_test, Alpha_target_test

In [ ]:
class DeepOPF(nn.Module):

  def __init__(self,inputSize,outputSize,hiddenSize1,hiddenSize2, nbhidden):
    super(DeepOPF, self).__init__()
    self.function1 = nn.Linear(inputSize,hiddenSize1)
    self.function2 = nn.Linear(hiddenSize1,hiddenSize2)
    self.function3 = nn.Linear(hiddenSize2,outputSize)
    self.nbhidden = nbhidden

  def forward(self,x):
    x = torch.relu(self.function1(x))
    for i in range(self.nbhidden):
      x = torch.relu(self.function2(x))
    x = torch.sigmoid(self.function3(x))
    return x

In [ ]:
def Parameters_init(ppc,PD_maxs, PD_mins, Power, slack_adapted):
  A= createAmatrix(ppc) #getting A matrix 
  Ones = np.zeros(shape=(np.shape(A)[0],1))
  A = A.astype(np.float32)
  A = torch.from_numpy(A) # A matrix as a tensor

  #getting ones for loss 2
  for x in range(len(Ones)):
    Ones[x] =1
  Ones = torch.from_numpy(Ones)

  #getting the difference for unnormalization
  PDdiff = []
  PDdifftorch = []
  it = 0
  for column in PD_maxs:
    PDdifftorch.append(PD_maxs[it]  - PD_mins[it] )
    it = it +1
  print(PDdifftorch)
  PDdiff = PDdifftorch.copy()
  PDdifftorch = torch.from_numpy(np.asarray(PDdifftorch, dtype = 'float64')) 
  print(PDdifftorch)

  # getting min for unnormalization 

  PDmintorch = torch.from_numpy(np.asarray(PD_mins, dtype = 'float64'))  

  #getting PG diff for getting PG from alphas

  PGdifftorch = []
  PGmin  = []
  it=0
  PGmax =  Power[:]["Pmax"]
  for column in PGmax:
      if (it != slack_adapted):
        PGdifftorch.append(Power.at[it, "Pmax"]- Power.at[it, "Pmin"])
        PGmin.append(Power.at[it,"Pmin"])
      it= it+1
  PGdifftorch = torch.from_numpy(np.asarray(PGdifftorch, dtype = 'float64'))

  #getting PG min for getting PG from alphas

  PGmintorch = torch.from_numpy(np.asarray(PGmin, dtype = 'float64'))

  #getting the B inverse for later use

  B_inverse = Get_B_Matrix_Inverse(slack_adapted,ppc)
  B_inverse = torch.from_numpy(B_inverse)
  return B_inverse, PGmintorch, PGdifftorch, PDmintorch, PDdifftorch, Ones, A , PDdiff

In [ ]:
def Train(slack_adapted, n_epochs, DataLoad,optimizer, Deep_OPF, B_inverse, PGmintorch, PGdifftorch, PDmintorch, PDdifftorch, Ones, A, PDdiff, unNormalizedPD_ws, PGdf_ws, ppc):
  from time import time
  t = time()
  for epochs in range(n_epochs):
    iteration = 0
    Loss_per_batch = []
    for i,(inputs,Alpha_target) in enumerate(DataLoad):
      optimizer.zero_grad()
      output_pred = Deep_OPF(inputs)
      loss1 = torch.nn.MSELoss(size_average=None, reduce=None, reduction = 'mean') #
      Loss1_input = loss1(output_pred ,Alpha_target) 
      #getting PG from output alpha
      PGOut0 = torch.mul(output_pred, PGdifftorch)
      PGOut0 = torch.add(PGOut0, PGmintorch)
      #getting PDs from input tensors 
      PDIn = torch.zeros(1,len(unNormalizedPD_ws.columns.to_list()))
      PGout = torch.zeros(1,len(unNormalizedPD_ws.columns.to_list()))
      columnOfInput = 0
      PDbuscolumn =0
      #adding the 0s
      for i in unNormalizedPD_ws:
        if (unNormalizedPD_ws[i].max() != 0):
          PDIn[0,PDbuscolumn]= inputs[0,columnOfInput]
          columnOfInput = columnOfInput + 1
        PDbuscolumn =   PDbuscolumn + 1
          #unNormalizing PD
      PDIn = torch.mul(PDIn, PDdifftorch)
      PDIn = torch.add(PDIn, PDmintorch)
      columnOfOutput = 0
      PGbuscolumn =0
      #getting theta tilda
      PG_gen_list = PGdf_ws.columns.tolist()
      for col in unNormalizedPD_ws.columns:
        if col in PG_gen_list:
          PGout[0,PGbuscolumn] = PGOut0[0,columnOfOutput]
          columnOfOutput = columnOfOutput + 1
        PGbuscolumn =   PGbuscolumn + 1
      # preparing PD and PG in per unit
      PDInFinal =  To_Per_Unit_Torch(PDIn,ppc)
      PGOutFinal = To_Per_Unit_Torch(PGout,ppc)
      PDInFinal = torch.transpose(PDInFinal, 0, 1)
      PGOutFinal = torch.transpose(PGOutFinal, 0, 1)
      Difference = torch.subtract(PGOutFinal,PDInFinal) #getting the difference between PG and PD
      Angles =torch.matmul(B_inverse,Difference)   #getting the angles in randiants / multiply by 180/pi for randiants
      length_withslack = 1+len(unNormalizedPD_ws.columns.to_list())
      Angles_hat = torch.zeros([length_withslack,1] )
      #getting angles with slack
      it=0
      for i in range(length_withslack):
        if i != slack_adapted:
        
          Angles_hat[i,0] = Angles[it,0]
          it = it +1
      
      #calculating loss 2
      Loss2_input = torch.matmul(A,Angles_hat)    
      Loss2_input = torch.square(Loss2_input)
      Loss2_input = torch.subtract(Loss2_input,Ones)
      Loss2_input = torch.mean(Loss2_input)
      Total_Loss = Loss1_input + Loss2_input* 0.00001
      Total_Loss.backward()#
      optimizer.step()#
    
    print(epochs, Loss1_input.item())
    

  print(f'Time to run: {(time() - t)} seconds')
  return
      

In [ ]:
def Test(inputs_test, Alpha_target_test, PD_test, Deep_OPF):
  from time import time
  from numpy import vstack
  import numpy as np
  import math

  from sklearn.metrics import accuracy_score
  test_data = []
  for i in range(len(PD_test)):
      test_data.append([inputs_test[i],Alpha_target_test[i]])
  TestLoad = torch.utils.data.DataLoader(dataset = test_data, batch_size = 64, shuffle = True)
  predictions, actuals = list(), list()
  t = time()
  for i, (inputs_test, Alpha_target_test) in enumerate(test_data):
      # evaluate the model on the test set
      yhat = Deep_OPF(inputs_test)
      # retrieve numpy array
      yhat = yhat.detach().numpy()
      actual = Alpha_target_test.numpy()
      predictions.append(yhat)
      actuals.append(actual)
  print(f'Time to run: {(time() - t)} seconds')
  return actuals, predictions

Average of Root Mean Squared error of 15000 testing simulations

In [ ]:
def AlphaRMSE(actuals, predictions):
  from sklearn.metrics import mean_squared_error
  n=[]
  actuals = np.array(actuals)
  predictions = np.array(predictions)
  for i in range(len(actuals[0])):
    s=mean_squared_error(actuals[i],predictions[i],squared=False)
    n.append(s)
  n=np.array(n)
  avg = n.sum()/len(n)
  return avg

Average R2 SCORE 

In [ ]:
def AlphaR2Score(actuals,predictions):
  from sklearn.metrics import r2_score
  n=[]
  actuals = np.array(actuals)
  predictions = np.array(predictions)
  for i in range(len(actuals[0])):
    c=r2_score(actuals[i], predictions[i])
    n.append(c)
  n=np.array(n) 
  avg = n.sum()/len(n) 
  return avg

Saving the model

In [ ]:
def SaveModel(Deep_OPF):
  PATH= "model.pth"
  torch.save(Deep_OPF.state_dict(), PATH)
  return PATH

Gettig the real Pg

In [ ]:
def GetPGsWithoutSlack(actuals,slack, Power, PGdf,predictions):
  actuals_df = pd.DataFrame(actuals)
  PG_actual_ws = GetPGfromAlphaws(actuals_df,Power,slack, PGdf)
  print("PG_actual_ws")
  print(PG_actual_ws)
  predictions_df = pd.DataFrame(predictions)
  PG_predicted_ws = GetPGfromAlphaws(predictions_df,Power,slack, PGdf)
  print("PG_predicted_ws")
  print(PG_predicted_ws)
  #PG_predicted_ws.to_csv(r'PG_predictions_ws.csv',index=False)
  return PG_predicted_ws,  PG_actual_ws

Getting the phase angles

In [ ]:
def GetAnglesWithoutSlack(PG_actual_ws, PD_test,PDdiff, unNormalizedPD_ws,PD_mins, PG_predicted_ws,slack_adapted,ppc):
  R = len(PD_test.index)
  C= len(unNormalizedPD_ws.columns)
  PD_test_unNormalized = np.zeros((R,C))
  PD_test = PD_test.to_numpy()
  columnOfInput = 0
  PDbuscolumn = 0
  for i in unNormalizedPD_ws:
        if (unNormalizedPD_ws[i].max() != 0):
          for row in range(R):
            PD_test_unNormalized[row,PDbuscolumn]= PD_test[row,columnOfInput]
            PD_test_unNormalized[row,PDbuscolumn] = PD_test_unNormalized[row,PDbuscolumn] * PDdiff[PDbuscolumn]
            PD_test_unNormalized[row,PDbuscolumn] = PD_test_unNormalized[row,PDbuscolumn] + PD_mins[PDbuscolumn]
          columnOfInput = columnOfInput + 1
        PDbuscolumn =   PDbuscolumn + 1

  PD_test_unNormalized =  pd.DataFrame(PD_test_unNormalized)
  #print(PD_test)
  #print(PD_test_unNormalized)
  #print(PG_actual_ws)
  angles_actual=Get_Phase_Angles(PG_actual_ws, PD_test_unNormalized,slack_adapted,ppc)
  angles_actual = pd.DataFrame(angles_actual)
  angles_actual = angles_actual.T 
  #print(angles_actual)
  #angles_actual.to_csv(r'Angles_actuals_ws.csv',index=False)

  angles_predicted=Get_Phase_Angles(PG_predicted_ws, PD_test_unNormalized,slack_adapted,ppc)
  angles_predicted = pd.DataFrame(angles_predicted)
  angles_predicted = angles_predicted.T 
  #print(angles_predicted)
  #angles_predicted.to_csv(r'Angles_predictions_ws.csv',index=False)
  return angles_actual , angles_predicted, PD_test_unNormalized

Average of Root Mean Squared error of 15000 Angles and PG Testing Results

In [ ]:
def RMSEPGAngles(PG_actual_ws,PG_predicted_ws,angles_actual,angles_predicted):
  from sklearn.metrics import mean_squared_error
  n1=[]
  n2=[]
  actualAngles = np.array(angles_actual)
  predictionAngles = np.array(angles_predicted)
  actualPG = np.array(PG_actual_ws)
  predictionPG = np.array(PG_predicted_ws)
  print(len(actualPG[0]))
  for i in range(len(actualPG[0])):
    s1=mean_squared_error(actualPG[i],predictionPG[i],squared=False)
    n1.append(s1)
  n1=np.array(n1)
  print("RMSE For PG:")
  #print(n1)
  print(n1.sum()/len(n1))
  avgP = n1.sum()/len(n1)
  for i in range(len(actualAngles[0])):
    s2=mean_squared_error(actualAngles[i],predictionAngles[i],squared=False)
    n2.append(s2)
  n2=np.array(n2)
  print("RMSE For Angles:")
  #print(n2)
  print(n2.sum()/len(n2))
  avgA = n2.sum()/len(n2)
  return avgA, avgP, actualPG, actualAngles, predictionPG, predictionAngles


Average R2 SCORE 

In [ ]:
def R2PGAngles(actualPG,predictionPG,actualAngles,predictionAngles):
  from sklearn.metrics import r2_score
  n1= []
  n2 = []
  for i in range(len(actualPG[0])):
    c1=r2_score(actualPG[i],predictionPG[i])
    n1.append(c1)
  n1=np.array(n1)

  print("R2 For PG:")
  print(n1)
  print(n1.sum()/len(n1))
  for i in range(len(actualAngles[0])):
    c2=r2_score(actualAngles[i],predictionAngles[i])
    n2.append(c2)
  print("R2 For Angles:")

  n2=np.array(n2) 
  #print(n2)
  print((n2.sum()/len(n2)))
  avgP = n1.sum()/len(n1)
  avgA = n2.sum()/len(n2)
  return avgA, avgP

Getting PGi of the slack bus is obtained by subtracting output of the other
buses from the total load

In [ ]:
 def GettingSlackPG(PG_predicted_ws, PD_test_unNormalized):
  import pandas as pd
  PG_predicted_ws = pd.DataFrame(PG_predicted_ws.copy())
  PD_test_unNormalized=pd.DataFrame(PD_test_unNormalized.copy())
  PD_test_unNormalized["Total D"] = PD_test_unNormalized.sum(axis=1)
  PG_predicted_ws["Total G"] = PG_predicted_ws.sum(axis=1)
  print(PD_test_unNormalized)
  print(PG_predicted_ws)
  #for column in PD_test_unNormalized[['Total D']]:
  #for column in PG_predicted_ws[['Total G']]:
  slack_generation=PD_test_unNormalized["Total D"] - PG_predicted_ws["Total G"]
  print(slack_generation)
  return slack_generation, PG_predicted_ws

insert in Pg without slack

In [ ]:
def AddingSlackToPG(PG_predicted_ws,slack_generation, slack_adapted,slack,ppc):
  #print(PG_predicted_ws)
  PG_predicted_ws.insert(slack_adapted,str(slack),slack_generation)
  #print(PG_predicted_ws)
  PG_predicted_ws = PG_predicted_ws.drop(["Total G"], axis=1)
  #print(PG_predicted_ws)
  columnnames= []
  for row in ppc["gen"]:
    columnnames.append(row[0])
  PredictedPG_WithSlack = pd.DataFrame(PG_predicted_ws.to_numpy(), columns=columnnames)
  #print(Predicted_WithSlack)
  return PredictedPG_WithSlack

In [ ]:
def AddingSlacktoAngles(predictionAngles, slack_adapted, slack,ppc):
  predictionAngles = pd.DataFrame(predictionAngles)
  slackdf = pd.DataFrame(0, index=range(len(predictionAngles)), columns=[str(slack)])
  predictionAngles.insert(slack_adapted,str(slack),slackdf)
  columnnames= []
  for row in ppc["bus"]:
    columnnames.append(row[0])
  PredictedAngles_WithSlack = pd.DataFrame(predictionAngles.to_numpy(), columns=columnnames)
  return PredictedAngles_WithSlack 

Cost Function


In [ ]:
def GenerationCost(PG,ppc):
  PGCost = PG.copy()
  PGCost = PGCost.to_numpy()
  for col in range(PGCost.shape[1]):
    PGCost[:,col] = (PGCost[:,col]) * (PGCost[:,col]) * ppc["gencost"][col][4] + PGCost[:,col] *  ppc["gencost"][col][5] + ppc["gencost"][col][6]
  PGCost= pd.DataFrame(PGCost)
  PGCost["Total Cost"] = PGCost.sum(axis=1)
  PGCostAvg = PGCost["Total Cost"].mean()
  return PGCost, PGCostAvg

In [ ]:
def TestingAndResults(PD_test, actuals, predictions , slack_adapted, slack, Power, ppc, Deep_OPF, PGdf, PDdiff, unNormalizedPD_ws, PD_mins):
  #AlphaRMSE = AlphaRMSE(actuals, predictions)
  from sklearn.metrics import mean_squared_error
  n1=[]
  actualstest = np.array(actuals.copy())
  predictionstest = np.array(predictions.copy())
  for i in range(len(actualstest[0])):
    s1=mean_squared_error(actualstest[i],predictionstest[i],squared=False)
    n1.append(s1)
  n1=np.array(n1)
  AlphaRMSE = n1.sum()/len(n1)


  AlphaR2 = AlphaR2Score(actuals,predictions)
  Path = SaveModel(Deep_OPF)
  PG_predicted_ws,  PG_actual_ws = GetPGsWithoutSlack(actuals, slack , Power, PGdf,predictions)
  angles_actual , angles_predicted, PD_test_unNormalized = GetAnglesWithoutSlack(PG_actual_ws,PD_test,PDdiff, unNormalizedPD_ws, PD_mins, PG_predicted_ws, slack_adapted,ppc)
  RMSETestingavgA, RMSETestingavgP, actualPG, actualAngles, predictionPG, predictionAngles = RMSEPGAngles(PG_actual_ws,PG_predicted_ws,angles_actual,angles_predicted)
  R2avgA, R2avgP = R2PGAngles(actualPG,predictionPG,actualAngles,predictionAngles)
  slack_generation, PG_predicted_ws = GettingSlackPG(PG_predicted_ws, PD_test_unNormalized)
  PredictedPG_WithSlack =  AddingSlackToPG(PG_predicted_ws,slack_generation, slack_adapted,slack,ppc)
  PredictedAngles_WithSlack = AddingSlacktoAngles(predictionAngles, slack_adapted, slack, ppc)
  slack_generation_real,PG_actual_ws  = GettingSlackPG(PG_actual_ws, PD_test_unNormalized)
  ActualPG_WithSlack =  AddingSlackToPG(PG_actual_ws,slack_generation_real, slack_adapted,slack,ppc)
  ActualAngles_WithSlack = AddingSlacktoAngles(actualAngles, slack_adapted, slack, ppc)
  PredictedPG_WithSlack.to_csv(r'PGPredictions.csv',index=False)
  PredictedAngles_WithSlack.to_csv(r'AnglesPredictions.csv',index=False)
  ActualPG_WithSlack.to_csv(r'ActualPGs.csv',index=False)
  ActualAngles_WithSlack.to_csv(r'ActualAngles.csv',index=False)
  return  PredictedPG_WithSlack,PredictedAngles_WithSlack, ActualAngles_WithSlack, ActualPG_WithSlack, Path, AlphaR2, AlphaRMSE, RMSETestingavgA, RMSETestingavgP,R2avgA, R2avgP

In [ ]:
def dnn(split_dataset, batch_size, epoch,learning_rate, hidden_layer,neurons):
  #initialize ppc
  from case import case
  ppc = case()
  test_size = split_dataset
  hiddenSize1 = neurons
  hiddenSize2 = neurons
  batch_size = batch_size
  learning_rate =learning_rate
  n_epochs = epoch
  nbhidden = hidden_layer
  #test_size = 0.5
  #inputSize = 20
  #outputSize = 5
  #hiddenSize1 = 16
  #hiddenSize2 = 16
  #batch_size = 64
  #learning_rate = 1e-3
  #n_epochs = 235
  print("Initialization")
  Power,slack, slack_adapted = Initialize(ppc)
  Bbus  = GetAdmittance(ppc)
  PDdf, PGdf, VAdf, unNormalizedPD, unNormalizedPG = InitializeDataframes() #don't forget to set the csvs before
  PD_maxs, PD_mins = GetMaxandMins(PDdf, slack_adapted)
  PDdf = normalize(PDdf)
  PDdf_ws, PGdf_ws, unNormalizedPD_ws = RemovingSlackFromPDPG(PDdf,PGdf,slack, unNormalizedPD)
  B_tilda = RemovingSlack(slack_adapted, Bbus) 
  PDdf_withoutzeros, Alpha, inputSize , outputSize = PreparingDnnData(PDdf_ws,PGdf,Power,slack)
  PD_train, PD_test, Alpha_train, Alpha_test = InitializeTrainTestData(test_size, PDdf_withoutzeros, Alpha) 
  inputs, Alpha_target = TrainingData(PD_train, Alpha_train) 
  inputs_test, Alpha_target_test = TestingData(PD_test, Alpha_test)
  Deep_OPF = DeepOPF(inputSize,outputSize,hiddenSize1,hiddenSize2,nbhidden)
  emp = []
  for i in range(len(PD_train)):
    emp.append([inputs[i],Alpha_target[i]])
  DataLoad = torch.utils.data.DataLoader(dataset = emp, batch_size = batch_size, shuffle = True)
  optimizer = torch.optim.SGD(Deep_OPF.parameters(), lr= learning_rate)
  B_inverse, PGmintorch, PGdifftorch, PDmintorch, PDdifftorch, Ones, A, PDdiff = Parameters_init(ppc,PD_maxs, PD_mins, Power, slack_adapted)
  Train(slack_adapted, n_epochs, DataLoad,optimizer, Deep_OPF, B_inverse, PGmintorch, PGdifftorch, PDmintorch, PDdifftorch, Ones, A, PDdiff, unNormalizedPD_ws, PGdf_ws, ppc)
  actuals, predictions = Test(inputs_test, Alpha_target_test, PD_test, Deep_OPF)
  print('done')
  PredictedPG_WithSlack,PredictedAngles_WithSlack, ActualAngles_WithSlack, ActualPG_WithSlack, Path, AlphaR2Test, AlphaRMSETest, RMSETestingavgA, RMSETestingavgP,R2avgA, R2avgP = TestingAndResults(PD_test, actuals, predictions , slack_adapted, slack, Power, ppc, Deep_OPF, PGdf, PDdiff, unNormalizedPD_ws, PD_mins)
  PGCost, PGCostAvg = GenerationCost(PredictedPG_WithSlack,ppc)
  PGCostAct , PGCostAvgActual = GenerationCost(ActualPG_WithSlack, ppc)
  PGCost.to_csv(r'PGCost.csv',index=False)
  #return actuals, predictions , slack_adapted, slack, ppc, Deep_OPF , Power , PGdf, PD_test, PDdiff, unNormalizedPD_ws, PD_mins
  return PGCostAvg ,  AlphaR2Test, AlphaRMSETest ,  PGCostAvgActual

In [ ]:
 PGCostAvg ,  AlphaR2, AlphaRMSE   ,  PGCostAvgActual= dnn(split_dataset = 0.5 , batch_size = 64,epoch = 230,learning_rate= 0.001, hidden_layer = 1 ,neurons = 16)
 print( PGCostAvg)
 print( PGCostAvgActual)
 print(AlphaR2)
 print( AlphaRMSE)



FLASK

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install werkzeug 

In [ ]:
from flask_ngrok import run_with_ngrok
import threading
from flask import Flask , render_template, request, send_file
app = Flask(__name__, template_folder='/content/Templates')

run_with_ngrok(app)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/training.html',methods = ['GET'])
def train():
    return render_template('training.html')

@app.route('/Team.html',methods = ['GET'])
def team():
    return render_template('Team.html')

@app.route('/index.html')
def backhome():
    return render_template('index.html')

@app.route('/terminal',methods = ['POST','GET'])
def terminal():
    if request.method == 'POST':
        #f1 = request.files['PD']
        #f1.save(f1.filename)
        #f2 = request.files['PG']
        #f2.save(f2.filename)
        #f3 = request.files['Va']
        #f3.save(f3.filename)
        #f4 = request.files['case']
        #f4.save(f4.filename)
        result = request.form
        '''Read the values from HTML file and set the values for training.''' 
        result = request.form
        split_dataset = result['split_dataset']
        project_name = result['project_name']
        batch_size = result['batch_size']
        epoch = result['epoch']
        learning_rate = result['learning_rate']
        hidden_layer = result['hidden_layer'] 
        neurons = result['neurons']
        #print(neurons)
        PGCostAvg ,  AlphaR2, AlphaRMSE = dnn(split_dataset = float(split_dataset), batch_size = int(batch_size),epoch = int(epoch),learning_rate= float(learning_rate), hidden_layer = int(hidden_layer),neurons = int(neurons))
        #threading.Thread(target=app.run).start()
        R2_test = AlphaR2 *100
        RMSE_test =  AlphaRMSE*100
        cost= PGCostAvg *100
        return render_template('terminal.html', result = result, RMSE_test=RMSE_test, R2_test = R2_test, cost = cost)

@app.route('/savedmodel')
def savedmodel():

  return render_template('result.html')



@app.route('/download')
def download_file():
    path = "/content/Templates/index.html"
    return send_file(path, as_attachment=True)
@app.route('/download')
def download_file_actualP():
    path = "/content/ActualPGs.csv"
    return send_file(path, as_attachment=True)
@app.route('/download')
def download_file_actualA():
    path = "/content/ActualAngles.csv"
    return send_file(path, as_attachment=True)
@app.route('/download')
def download_file_predictedA():
    path = "/content/AnglesPredictions.csv"
    return send_file(path, as_attachment=True)
@app.route('/download')
def download_file_predictedP():
    path = "/content/PGPredictions.csv"
    return send_file(path, as_attachment=True)
@app.route('/download')
def download_filePD():
    path = "/content/IndexTemp/PDTemp.csv"
    return send_file(path, as_attachment=True)
@app.route('/download')
def download_filePG():
    path = "/content/IndexTemp/PGTemp.csv"
    return send_file(path, as_attachment=True)
@app.route('/download')
def download_fileVa():
    path = "/content/IndexTemp/VaTemp.csv"
    return send_file(path, as_attachment=True)
@app.route('/download')
def download_fileCase():
   path = "/content/IndexTemp/case.py"
   return send_file(path, as_attachment=True)
@app.route('/download')
def download_filePDPredict():
   path = "/content/IndexTemp/PDpredict.csv"
   return send_file(path, as_attachment=True)
@app.route('/download')
def download_file_model():
   path = "/content/model.pth"
   return send_file(path, as_attachment=True)
@app.route('/download')
def download_file_cost():
  path = "/content/PGCost.csv"
  return send_file(path, as_attachment=True)

@app.route('/pre-trained',methods = ['POST','GET'])
def pretrained():
  if request.method == 'POST':
     f1 = request.files['PD']
     f1.save(f1.filename)
     case = request.form['Pre-trained Model']
     result30 = { 'project_name' : 'IEEE Case 30','split_dataset' : 0.5 , 'batch_size' : 64, 'epoch' : 235 , 'learning_rate' : 0.001,'hidden_layer': 2, 'neurons': 16  }
     
     result57 = { 'project_name' : 'IEEE Case 57','split_dataset' : 0.5 , 'batch_size' : 64, 'epoch' : 235 , 'learning_rate' : 0.001,'hidden_layer': 4, 'neurons': 32  }
     
     result118 = { 'project_name' : 'IEEE Case 30','split_dataset' : 0.5 , 'batch_size' : 64, 'epoch' : 235 , 'learning_rate' : 0.001,'hidden_layer': 6, 'neurons': 64  }
     
     result300 = { 'project_name' : 'IEEE Case 30','split_dataset' : 0.5 , 'batch_size' : 64, 'epoch' : 235 , 'learning_rate' : 0.001,'hidden_layer': 6, 'neurons': 128  }
     
     if (str(case) == 'IEEE Case 30'):
       print('30')
       #PGCostAvg ,  AlphaR2, AlphaRMSE = dnnforpretrained(30)
       #R2_test = AlphaR2 *100
       #RMSE_test =  AlphaRMSE*100
       #cost= PGCostAvg *100
       return render_template('result.html', result = result30) # , RMSE_test=RMSE_test, R2_test = R2_test, cost = cost)
     ` 
     elif (str(case) == 'IEEE Case 57'):
        print('57')
        PGCostAvg ,  AlphaR2, AlphaRMSE = dnnforpretrained(57)
        R2_test = AlphaR2 *100
        RMSE_test =  AlphaRMSE*100
        cost= PGCostAvg *100
        return render_template('result.html', result = result57 , RMSE_test=RMSE_test, R2_test = R2_test, cost = cost)
      
      elif (str(case)== 'IEEE Case 118'):
        print('118')
        PGCostAvg ,  AlphaR2, AlphaRMSE = dnnforpretrained(118)
        R2_test = AlphaR2 *100
        RMSE_test =  AlphaRMSE*100
        cost= PGCostAvg *100
        return render_template('result.html', result = result118 , RMSE_test=RMSE_test, R2_test = R2_test, cost = cost)

      if (str(case) == 'IEEE Case 300'):
       PGCostAvg ,  AlphaR2, AlphaRMSE = dnnforpretrained(118)
       R2_test = AlphaR2 *100
       RMSE_test =  AlphaRMSE*100
       cost= PGCostAvg *100
       return render_template('result.html', result = result300 , RMSE_test=RMSE_test, R2_test = R2_test, cost = cost)
       

app.run()
   

IndentationError: ignored

Checking for first PGi constraint violations

Checking for second Thetha constraint violation 

In [ ]:
slack_generation, PG_predicted_ws = GettingSlackPG(PG_predicted_ws, PD_test_unNormalized)
PredictedPG_WithSlack =  AddingSlackToPG(PG_predicted_ws,slack_generation, slack_adapted,ppc)
PredictedAngles_WithSlack = AddingSlacktoAngles(predictionAngles, slack_adapted, slack, ppc)
slack_generation_real,PG_actual_ws  = GettingSlackPG(PG_actual_ws, PD_test_unNormalized)
ActualPG_WithSlack =  AddingSlackToPG(PG_actual_ws,slack_generation_real, slack_adapted,ppc)
ActualAngles_WithSlack = AddingSlacktoAngles(actualAngles, slack_adapted, slack, ppc)
PredictedPG_WithSlack.to_csv(r'PGPredictions.csv',index=False)
PredictedAngles_WithSlack.to_csv(r'AnglesPredictions.csv',index=False)
ActualPG_WithSlack.to_csv(r'ActualPGs.csv',index=False)
ActualAngles_WithSlack.to_csv(r'ActualAngles.csv',index=False)
